In [3]:
import pandas as pd
df = pd.read_csv("Daily Household Transactions.csv")
print(df.head())
print(df.info())

        Date                   Mode        Category              Subcategory  \
0  9/20/2018                   Cash  Transportation                    Train   
1  9/20/2018                   Cash            Food                   snacks   
2  9/19/2018  Saving Bank account 1    subscription                  Netflix   
3  9/17/2018  Saving Bank account 1    subscription  Mobile Service Provider   
4  9/16/2018                   Cash       Festivals             Ganesh Pujan   

                          Note  Amount Income/Expense Currency  
0         2 Place 5 to Place 0    30.0        Expense      INR  
1  Idli medu Vada mix 2 plates    60.0        Expense      INR  
2         1 month subscription   199.0        Expense      INR  
3            Data booster pack    19.0        Expense      INR  
4                  Ganesh idol   251.0        Expense      INR  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 8 columns):
 #   Column          No

In [5]:
import numpy as np
print("Original shape:", df.shape)
df.columns = df.columns.str.strip()
print("\nMissing values before cleaning:\n", df.isnull().sum())
text_cols = ['Mode', 'Category', 'Note', 'Income/Expense', 'Currency']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")
if 'Subcategory' in df.columns:
    df['Subcategory'] = df['Subcategory'].apply(
        lambda x: str(x).strip().title() if pd.notna(x) and str(x).strip() != "" else "Unknown"
    )
else:
    print(" No column named 'Subcategory'. Please check column names above.")
df = df.dropna(subset=['Date', 'Amount'])
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
def clean_text(x):
    if pd.isna(x) or str(x).strip() == "":
        return "Unknown"
    return str(x).strip().title()

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

df['Income/Expense'] = df['Income/Expense'].replace({
    'Expenses': 'Expense',
    'Incomes': 'Income'
}).str.capitalize()

df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df = df.dropna(subset=['Amount'])
df = df[df['Amount'] >= 0]

df = df.drop_duplicates()
Q1 = df['Amount'].quantile(0.25)
Q3 = df['Amount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['Amount'] >= lower_bound) & (df['Amount'] <= upper_bound)]
df = df.sort_values(by='Date').reset_index(drop=True)
df.to_csv("Daily_Household_Transactions_Cleaned.csv", index=False)

print("\n Data cleaning & normalization complete!")
print("New shape:", df.shape)
print("\nSample after cleaning:\n", df.head())


Original shape: (2461, 8)

Missing values before cleaning:
 Date                0
Mode                0
Category            0
Subcategory       635
Note              521
Amount              0
Income/Expense      0
Currency            0
dtype: int64

 Data cleaning & normalization complete!
New shape: (2135, 8)

Sample after cleaning:
          Date         Mode        Category Subcategory  \
0  2015-01-01         Cash  Transportation     Unknown   
1  2015-01-01  Credit Card            Food     Unknown   
2  2015-01-01         Cash  Transportation     Unknown   
3  2015-01-01         Cash  Transportation     Unknown   
4  2015-01-01         Cash         Culture     Unknown   

                                   Note  Amount Income/Expense Currency  
0  Share Auto - Hospital To Brc Station    10.0        Expense      Inr  
1                Bendys Chicken Biryani   400.0        Expense      Inr  
2      Share Jeep - Place T Top To Base    20.0        Expense      Inr  
3           Share 

In [6]:
import pandas as pd
from prophet import Prophet
df = pd.read_csv("Daily_Household_Transactions_Cleaned.csv")
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df = df[df['Income/Expense'].str.lower() == 'expense']

monthly = df.groupby(pd.Grouper(key='Date', freq='M'))['Amount'].sum().reset_index()
monthly.columns = ['ds', 'y']

monthly.to_csv("monthly.csv", index=False)
print(" Saved monthly aggregated totals as monthly.csv")
print(monthly.tail())

model = Prophet(yearly_seasonality=True, daily_seasonality=False)
model.fit(monthly)

future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

last_actual_date = monthly['ds'].max()
future_forecast = forecast[forecast['ds'] > last_actual_date][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

future_forecast.to_csv("forecast.csv", index=False)
print("\n Saved 12-month ahead forecast as forecast.csv")
print(future_forecast)

full_forecast = forecast[['ds', 'yhat']].merge(monthly, on='ds', how='left')
full_forecast.to_csv("forecast_with_history.csv", index=False)


/tmp/ipython-input-2989239665.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.groupby(pd.Grouper(key='Date', freq='M'))['Amount'].sum().reset_index()


 Saved monthly aggregated totals as monthly.csv
           ds         y
40 2018-05-31   9170.00
41 2018-06-30   8782.02
42 2018-07-31  19988.36
43 2018-08-31  11595.65
44 2018-09-30   2724.00


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp99l876h/1_yxbw89.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp99l876h/xyej_8tg.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99270', 'data', 'file=/tmp/tmpp99l876h/1_yxbw89.json', 'init=/tmp/tmpp99l876h/xyej_8tg.json', 'output', 'file=/tmp/tmpp99l876h/prophet_modeldm1o4m14/prophet_model-20250814070406.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:04:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:04:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future versio


 Saved 12-month ahead forecast as forecast.csv
           ds          yhat    yhat_lower    yhat_upper
45 2018-10-31  15183.127939  11677.067997  18796.737742
46 2018-11-30  13158.501733   9535.730949  16732.230021
47 2018-12-31  16002.217267  12443.967344  19555.295091
48 2019-01-31  14031.540015  10667.766650  17521.267315
49 2019-02-28  12186.441165   9084.957142  15571.073280
50 2019-03-31  18110.537441  14521.661490  21461.162263
51 2019-04-30  13027.376267   9542.117045  16740.086027
52 2019-05-31  13203.828904   9547.603565  16581.967288
53 2019-06-30  10823.305191   7078.878326  14486.451506
54 2019-07-31  16045.027491  12688.541158  19422.989203
55 2019-08-31  14174.789110  10579.153374  17616.891408
56 2019-09-30   9932.161715   6556.388558  13181.629350


In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv("Daily_Household_Transactions_Cleaned.csv")
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df[df['Income/Expense'].str.lower() == 'expense']

monthly = df.groupby(pd.Grouper(key='Date', freq='M'))['Amount'].sum().reset_index()
monthly.columns = ['ds', 'y']

avg_monthly_expense = monthly['y'].mean()
std_monthly_expense = monthly['y'].std()

k_best_case = 0.5
k_expected = 1.0
k_worst_case = 1.5

budget_best_case = avg_monthly_expense + k_best_case * std_monthly_expense
budget_expected = avg_monthly_expense + k_expected * std_monthly_expense
budget_worst_case = avg_monthly_expense + k_worst_case * std_monthly_expense

budget_df = pd.DataFrame({
    'Scenario': ['Best Case', 'Expected', 'Worst Case'],
    'Variance_Multiplier': [k_best_case, k_expected, k_worst_case],
    'Budget': [budget_best_case, budget_expected, budget_worst_case]
})

print("Historical Monthly Expense Budget Scenarios:")
print(budget_df)
print(f"\nHistorical average monthly expense : ₹{avg_monthly_expense:,.2f}")
print(f"Historical monthly expense std dev  : ₹{std_monthly_expense:,.2f}")


Historical Monthly Expense Budget Scenarios:
     Scenario  Variance_Multiplier       Budget
0   Best Case                  0.5  12192.09481
1    Expected                  1.0  14063.65162
2  Worst Case                  1.5  15935.20843

Historical average monthly expense : ₹10,320.54
Historical monthly expense std dev  : ₹3,743.11


/tmp/ipython-input-1688373433.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.groupby(pd.Grouper(key='Date', freq='M'))['Amount'].sum().reset_index()


In [8]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [9]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="Forecast vs Actual Dashboard", layout="wide")

st.title(" Monthly Expense: Forecast vs Actual Spend")

monthly = pd.read_csv("monthly.csv")
forecast = pd.read_csv("forecast.csv")

df = pd.merge(monthly, forecast[['ds','yhat']], on='ds', how='outer').sort_values('ds')

fig = px.line(df, x="ds", y=["y", "yhat"],
              labels={"value": "Amount (₹)", "ds": "Month"},
              title="Actual vs Forecast Monthly Expenses")
st.plotly_chart(fig, use_container_width=True)

avg_expense = df['y'].mean()
st.metric(label="Average Monthly Expense (Actual)", value=f"₹{avg_expense:,.2f}")

Writing app.py


In [11]:
# Install dependencies
!pip install pyngrok streamlit -q

from pyngrok import ngrok
import time
import threading
import os

# Kill any existing tunnels
ngrok.kill()

# Authenticate ngrok (replace with your own token from https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("31FLyj5XxRbQKELoteSfkIoFNTv_5XsBCpBm8BGTBykLVZVKq")

# Start the Streamlit app in the background
def run_app():
    os.system("streamlit run app.py --server.port 8501")

threading.Thread(target=run_app).start()

# Wait for app to start
time.sleep(5)

# Create ngrok tunnel to port 8501
public_url = ngrok.connect(8501)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://92129da45f03.ngrok-free.app" -> "http://localhost:8501"
